In [2]:
import cv2
import numpy as np
import os

In [3]:
def sharpening(imgArr):
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    return cv2.filter2D(imgArr, -1, kernel)

In [4]:
def erode(imgArr, kernelSize=5):    
    kernel = np.ones((kernelSize, kernelSize),np.uint8)
    return cv2.dilate(imgArr,kernel,iterations = 1)

In [5]:
def dilate(imgArr, kernelSize=5):
    kernel = np.ones((kernelSize, kernelSize),np.uint8)
    return cv2.erode(imgArr,kernel,iterations = 1)

In [126]:
def opening(imgArr, kernelSize=5):
    return erode(dilate(imgArr, kernelSize), kernelSize)

In [127]:
def closing(imgArr, kernelSize=5):
    return dilate(erode(imgArr, kernelSize), kernelSize)

In [6]:
def binaryThresh(imgArr, threshold=175, maxVal=255):    
    ret,th = cv2.threshold(imgArr, threshold, 255, cv2.THRESH_BINARY)
    return th

In [232]:
def binaryInRange(imgArr, minBound, upperBound):    
    return cv2.inRange(imgArr, minBound, upperBound)

In [7]:
def binaryOtsu(imgArr):    
    ret,th = cv2.threshold(imgArr, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return th

In [72]:
def binaryAdaptiveMean(imgArr):
    return cv2.adaptiveThreshold(imgArr,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,9,4)

In [73]:
def binaryAdaptiveGauss(imgArr):
    return cv2.adaptiveThreshold(imgArr,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,9,4)

In [60]:
def binarySauvola(srcPath, targetPath): 
    os.system("convert "+srcPath+" -depth 8 "+targetPath+".pgm")
    os.system("./binarize-sauvola -i "+targetPath+".pgm -o "+targetPath+".pgm")
    os.system("convert "+targetPath+".pgm -depth 8 "+targetPath)
    os.system("rm "+targetPath+".pgm")

In [10]:
def medianBlur(imgArr):
    return cv2.medianBlur(imgArr,5)

In [501]:
def removeEdges(imgArr):
    imgArr = imgArr.copy()
    vertical   = vertProjection(imgArr)
    horizontal = horizProjection(imgArr)    
    
    numRows    = len(vertical)
    numColumns = len(horizontal)
    
    horizPositions  = []
    for horizIdx in range(0, len(horizontal)):
        horizValue = horizontal[horizIdx]
        if horizValue / numRows < 0.30:
            horizPositions.append(horizIdx)            
    
    topPosition = getLeftTopBorderEdge(imgArr, horizPositions)
    botPosition = getRightBotBorderEdge(imgArr, horizPositions)
    for horizIdx in range(0, topPosition):
        for vertIdx in range(0, numRows):
            imgArr[horizIdx][vertIdx] = 255   
    for horizIdx in range(botPosition, numColumns):
        for vertIdx in range(0, numRows):
            imgArr[horizIdx][vertIdx] = 255  
        
    vertPositions  = []
    for vertIdx in range(0, len(vertical)):
        vertValue = vertical[vertIdx]
        if vertValue / numColumns < 0.30:
            vertPositions.append(vertIdx)

    leftPosition  = getLeftTopBorderEdge(imgArr, vertPositions) 
    rightPosition = getRightBotBorderEdge(imgArr, vertPositions)
    if(leftPosition > rightPosition):
        leftPosition = 0
        
    for horizIdx in range(0, numColumns):
        for vertIdx in range(0, leftPosition):
            imgArr[horizIdx][vertIdx] = 255   
        for vertIdx in range(rightPosition, numRows):
            imgArr[horizIdx][vertIdx] = 255           
    
    return imgArr

In [12]:
def getRightBotBorderEdge(imgArr, positions):
    maxInterrupt = 0
    if(len(positions) == 0):
        return -1
    lastPosition = positions[-1]
    positions = reversed(positions)
    for position in positions:
        maxInterrupt = lastPosition - position
        if(maxInterrupt > 20):
            return lastPosition
        lastPosition = position
    return lastPosition

In [13]:
def getLeftTopBorderEdge(imgArr, positions):
    maxInterrupt = 0
    if(len(positions) == 0):
        return 0
    lastPosition = positions[0]
    for position in positions:
        maxInterrupt = position - lastPosition
        if(maxInterrupt > 20):
            return lastPosition
        lastPosition = position
    return lastPosition

In [14]:
def vertProjection(imgArr):
    imgArr = imgArr.astype(np.bool)
    return np.sum(imgArr, axis=0)

In [15]:
def horizProjection(imgArr):
    imgArr = imgArr.astype(np.bool)
    return np.sum(imgArr, axis=1)

In [260]:
def binarizationFaulty(imgArr):
    blackPixels = 0
    whitePixels = 0
    unique, counts = np.unique(imgArr, return_counts=True)
    results = dict(zip(unique, counts))
    if(0 in results):
        blackPixels = results[0]
    if(255 in results):
        whitePixels = results[255]
    total = max(blackPixels + whitePixels, 1)
    return  blackPixels / total > 0.8 or \
            whitePixels / total > 0.90

In [499]:
def binarizeImage(srcPath, targetPath):
    print('IMG ' + srcPath)
    imgArr = cv2.imread(srcPath, 0) 
    
    edges = binaryEdges(imgArr)
    fixed = removeEdges(edges)
    if(not binarizationFaulty(fixed)):
        cv2.imwrite(targetPath, fixed)    
        print('fixed')
        return
    
    cv2.imwrite(targetPath, edges)    
    print('edges')

In [463]:
def binaryEdges(imgArr):
    ret, thresh = cv2.threshold(imgArr, 127, 255, 0)
    imgArr, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    imgArr = cv2.drawContours(imgArr, contours, 0, (255), 1)
    return imgArr

In [474]:
def binarizeAll(srcPath, targetPath):    
    for f in os.listdir(srcPath):
        fileName = f.replace('.jpg', '')
        binarizeImage(srcPath+fileName+'.jpg', targetPath+fileName+'.tif')

In [502]:
binarizeAll(  '/home/bnu/Desktop/data/input/',
                '/home/bnu/Documents/CODE/Output/')

IMG /home/bnu/Desktop/data/input/2209_vollst_Büsching_0190_00000207.jpg
edges
IMG /home/bnu/Desktop/data/input/0004_grimm_donner_1855_0004.jpg
edges
IMG /home/bnu/Desktop/data/input/0001_1433243313511_1_221_Seite_1_Bild_0000.jpg
edges
IMG /home/bnu/Desktop/data/input/1213 Fauriel Origine_0009_00000009.jpg
edges
IMG /home/bnu/Desktop/data/input/2174_Haimonskinder_0006_00000006.jpg
edges
IMG /home/bnu/Desktop/data/input/2227_vollst_Temme_0003_00000003.jpg
edges
IMG /home/bnu/Desktop/data/input/0002_1433243313511_1_221_Seite_1_Bild_0001.jpg
fixed
IMG /home/bnu/Desktop/data/input/2266_vollst_Sommer_0100_image_20171218_124902.jpg
edges
IMG /home/bnu/Desktop/data/input/00000173.jpg
fixed
IMG /home/bnu/Desktop/data/input/2209_vollst_Büsching_0010_00000011.jpg
fixed
IMG /home/bnu/Desktop/data/input/2810 Ulr v Lichtenstein_0008_00000008.jpg
edges
IMG /home/bnu/Desktop/data/input/1433243313511_1_16a.jpg
edges
IMG /home/bnu/Desktop/data/input/2227_vollst_Temme_0025_00000024.jpg
edges
IMG /home/bn